# Titanic

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier 
from sklearn.datasets import make_classification
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import GaussianNB

In [2]:
test = pd.read_csv("Data/test.csv")
train = pd.read_csv("Data/train.csv")

In [3]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## Priprava podatkov

In [4]:
train = train[np.invert(np.isnan(train["Age"]))]
test["Age"] = test["Age"].fillna(np.mean(test["Age"]))
test["Fare"] = test["Fare"].fillna(np.mean(test["Fare"]))
#test = test[np.invert(np.isnan(test["Age"]))]
#test = test[np.invert(np.isnan(test["Fare"]))]

Nov atribut minor:

In [5]:
def minors(x):
    minor = []
    for a in x["Age"]:
        if a < 16:
            minor = np.append(minor, 1)
        else:
            minor = np.append(minor, 0)
    x["Minor"] = minor

In [6]:
minors(train)
minors(test)

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


Nova atributa na podlagi priimkov:
* Vsaj en prezivel z istim priimkom
* Vsi umrli z istim priimkom

In [7]:
def allDied(x, y, ista):
    allD = []
    lived = []
    allLived = []
    oneDead = []
    for index, row in x.iterrows():
        surname = row["Name"].split(",")[0]
        dead = 0
        alive = 0
        for index2, row2 in y.iterrows():
            if ista:
                if index != index2:
                    surname2 = row2["Name"].split(",")[0]
                    if surname2 == surname:
                        if row2["Survived"] == 1:
                            alive += 1
                        else:
                            dead += 1
            else:
                surname2 = row2["Name"].split(",")[0]
                if surname2 == surname:
                    if row2["Survived"] == 1:
                        alive += 1
                    else:
                        dead += 1
        if dead + alive < 2:
            allD = np.append(allD, 0)
            lived = np.append(lived, 0)
            allLived = np.append(allLived, 0)
            oneDead = np.append(oneDead, 0)
        else:
            if alive == 0:
                allD = np.append(allD, 1)
            else:
                allD = np.append(allD, 0)
            if alive >= 1:
                lived = np.append(lived, 1)
            else:
                lived = np.append(lived, 0)
            if dead == 0:
                allLived = np.append(allLived, 1)
            else:
                allLived = np.append(allLived, 0)
            if dead >= 1:
                oneDead = np.append(oneDead, 1)
            else:
                oneDead = np.append(oneDead, 0)
    x["SurnameAllDied"] = allD
    x["SurnameOneLived"] = lived
    x["SurnameOneDead"] = oneDead
    x["SurnameAllLived"] = allLived

In [8]:
allDied(train, train, True)
allDied(test, train, False)

## Pretvorba cabins

In [9]:
palube = []
for a in train["Cabin"]:
    if not pd.isna(a):
        if a[0] not in palube:
            palube = np.append(palube, a[0])

In [10]:
palube = []
for a in train["Cabin"]:
    if not pd.isna(a):
        palube = np.append(palube, a[0])
    else:
        palube = np.append(palube, a)
train["Deck"] = palube

In [11]:
palube = []
for a in test["Cabin"]:
    if not pd.isna(a):
        palube = np.append(palube, a[0])
    else:
        palube = np.append(palube, a)
test["Deck"] = palube

Tabele za napovedovanje:

In [78]:
def one_hot(name, x, y, bol):
    category = np.array([])
    for a in y[name]:
        if not pd.isna(a):
            if a not in category:
                category = np.append(category, a)
    category = np.sort(category)
    for c in category:
        if not pd.isna(c):
            current = []
            for a in y[name]:
                if a == c:
                    current = np.append(current, 1)
                else:
                    current = np.append(current, 0)
            if bol:
                f = name + ":" + str(c)
                x[f] = current
            else:
                x[c] = current

In [79]:
y = train["Survived"]
x = train[["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived", "SurnameAllLived", "SurnameOneDead", "SibSp", "Parch"]]
x["FamilySize"] = train["Parch"] + train["SibSp"]
one_hot("Sex", x, train, False)
one_hot("Deck", x, train, False)
one_hot("Embarked", x, train, True)
x = x.loc[:, x.columns != 'nan']
#enc = OneHotEncoder()
#enc.fit(train)

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/tmp/ipykernel_5244/4009306914.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if a not in category:


In [80]:
xtest = test[["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived", "SibSp", "Parch"]]
xtest["FamilySize"] = test["Parch"] + test["SibSp"]
one_hot("Sex", xtest, test, False)
one_hot("Deck", xtest, test, False)
xtest = xtest.loc[:, xtest.columns != 'nan']
one_hot("Embarked", xtest, test, True)

/tmp/ipykernel_5244/4009306914.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if a not in category:


In [81]:
x

,Pclass,Age,Fare,Minor,SurnameAllDied,SurnameOneLived,SurnameAllLived,SurnameOneDead,SibSp,Parch,...,B,C,D,E,F,G,T,Embarked:C,Embarked:Q,Embarked:S
0,3,22.0,7.2500,0.0,0.0,0.0,0.0,0.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,38.0,71.2833,0.0,0.0,0.0,0.0,0.0,1,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,26.0,7.9250,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,35.0,53.1000,0.0,0.0,0.0,0.0,0.0,1,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,3,35.0,8.0500,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,3,39.0,29.1250,0.0,1.0,0.0,0.0,1.0,0,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
886,2,27.0,13.0000,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
887,1,19.0,30.0000,0.0,0.0,1.0,0.0,1.0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
889,1,26.0,30.0000,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Ocenjevanje atributov

In [17]:
from sklearn.feature_selection import mutual_info_classif

In [82]:
trenutni = mutual_info_classif(x,y)
for a in range(99):
    trenutni += mutual_info_classif(x,y)
trenutni/100

array([0.06567035, 0.02442944, 0.13330875, 0.01064883, 0.02839469,
       0.011579  , 0.00683772, 0.00730782, 0.01679371, 0.01921533,
       0.04128441, 0.15014397, 0.15028413, 0.00652137, 0.02217525,
       0.00982339, 0.01556692, 0.01314412, 0.00783589, 0.0073249 ,
       0.00611563, 0.02006755, 0.00643882, 0.01584119])

In [83]:
for a, b in sorted(zip(trenutni, x.columns)):
    print(b + ": " + str(a / 100))

T: 0.006115633009123893
Embarked:Q: 0.0064388150139543625
A: 0.006521366645027826
SurnameAllLived: 0.006837723087089302
SurnameOneDead: 0.007307818560027621
G: 0.007324898463843743
F: 0.007835892192531866
C: 0.009823386665417875
Minor: 0.010648832400358764
SurnameOneLived: 0.011578997485261932
E: 0.013144123006992543
D: 0.015566916238129527
Embarked:S: 0.01584119259520008
SibSp: 0.016793709118710445
Parch: 0.019215326688227063
Embarked:C: 0.020067545222727093
B: 0.022175252171258642
Age: 0.024429444920752653
SurnameAllDied: 0.028394685979470785
FamilySize: 0.04128440886726113
Pclass: 0.06567034553067
Fare: 0.1333087469420917
female: 0.15014397009956926
male: 0.15028413495952395


## Napovedovanje

### Random forest

In [88]:
#x3 = x[["Minor", "SurnameAllDied", "female"]]
#x3 = x[['Minor', 'SurnameAllDied', 'FamilySize', 'female', 'male', 'E']]
#x3 = x[['Minor', 'SurnameAllDied', 'FamilySize', 'male', 'E']] 0,77272
#x3 = x[['Age', 'Minor', 'SurnameAllDied', 'male', 'D', 'E', 'Embarked:S']] 0,71770
x3 = x[['Pclass', 'Age', 'Minor', 'SurnameAllDied', 'FamilySize', 'female', 'male', 'Embarked:S']]
x3test = xtest[['Pclass', 'Age', 'Minor', 'SurnameAllDied', 'FamilySize', 'female', 'male', 'Embarked:S']]
clf = RandomForestClassifier()
clf.fit(x3,y)
r = clf.predict(x3test)

In [89]:
resoult = test[["PassengerId"]]
resoult["Survived"] = r
resoult.to_csv("Predictions/rfc42.csv",  index=False)

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


### Naive Bayes

In [62]:
#x3 = x[["Minor", "SurnameAllDied", "female"]]
#x3 = x[['Minor', 'SurnameAllDied', 'FamilySize', 'female', 'male', 'E']]
#x3 = x[['Minor', 'SurnameAllDied', 'FamilySize', 'male', 'E']] 0,7751
#x3 = x[['Age', 'Minor', 'SurnameAllDied', 'FamilySize', 'female', 'E']] 0,7751
x3 = x[['Pclass', 'Age', 'Minor', 'SurnameAllDied', 'female', 'male', 'E']]
x3test = xtest[['Pclass', 'Age', 'Minor', 'SurnameAllDied', 'female', 'male', 'E']]
gnb = GaussianNB()
gnb.fit(x3,y)
tt = gnb.predict(x3test)

In [49]:
x

,Minor,SurnameAllDied,FamilySize,female,male,E
0,0.0,0.0,1,0.0,1.0,0.0
1,0.0,0.0,1,1.0,0.0,0.0
2,0.0,0.0,0,1.0,0.0,0.0
3,0.0,0.0,1,1.0,0.0,0.0
4,0.0,0.0,0,0.0,1.0,0.0
...,...,...,...,...,...,...
885,0.0,1.0,5,1.0,0.0,0.0
886,0.0,0.0,0,0.0,1.0,0.0
887,0.0,0.0,0,1.0,0.0,0.0
889,0.0,0.0,0,0.0,1.0,0.0


In [63]:
resoult2 = test[["PassengerId"]]
resoult2["Survived"] = tt
#resoult2 == resoult
resoult2.to_csv("Predictions/nbc4.csv",  index=False)

/home/matej/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


## Napovedovanje + testiranje

### Random forest

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from itertools import combinations

In [26]:
maxS = 0.65
maxT = np.array([])
for s in range(4,11):
    for a in combinations(["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived", "FamilySize", "female", "male", "A", "E", "F"], s):
        x1 = x[np.array(a)]
        ca = 0
        for i in range(7):
            x_train, x_test, y_train, y_test = train_test_split(x1, y, test_size=0.33)
            clf = RandomForestClassifier()
            clf.fit(x_train, y_train)
            r = clf.predict(x_test)
            ca += accuracy_score(y_test, r)
        current = ca / 7
        if maxS < current:
            maxS = current
            maxT = a
        #print(current)
    print(s)
    print(maxT)
    print(maxS)
    maxS = 0.65
    maxT = np.array([])

4
('Minor', 'SurnameAllDied', 'female', 'F')
0.8462469733656174


KeyboardInterrupt: 

In [140]:
maxS

0.8401937046004841

### Naive bayes

In [85]:
maxS = 0.65
maxT = np.array([])
for s in range(4,11):
    for a in combinations(["Pclass", "Age", "Fare", "Minor", "SurnameAllDied", "SurnameOneLived", "FamilySize", "female", "male", "B", "D", "E", "Embarked:C", "Embarked:S"], s):
        x1 = x[np.array(a)]
        ca = 0
        for i in range(10):
            x_train, x_test, y_train, y_test = train_test_split(x1, y, test_size=0.33)
            gnb = GaussianNB()
            gnb.fit(x_train, y_train)
            r = gnb.predict(x_test)
            ca += accuracy_score(y_test, r)
        current = ca / 10
        if maxS < current:
            maxS = current
            maxT = a
        #print(current)
    print(s)
    print(maxT)
    print(maxS)
    maxS = 0.65
    maxT = np.array([])

4
('Minor', 'SurnameAllDied', 'female', 'E')
0.8317796610169491
5
('Age', 'Minor', 'SurnameAllDied', 'male', 'E')
0.8381355932203389
6
('Age', 'Minor', 'SurnameAllDied', 'FamilySize', 'female', 'male')
0.8419491525423728
7
('Age', 'Minor', 'SurnameAllDied', 'male', 'D', 'E', 'Embarked:S')
0.83135593220339
8
('Pclass', 'Age', 'Minor', 'SurnameAllDied', 'FamilySize', 'female', 'male', 'Embarked:S')
0.8317796610169491
9
('Age', 'Minor', 'SurnameAllDied', 'FamilySize', 'female', 'male', 'B', 'E', 'Embarked:C')
0.8249999999999998
10
('Pclass', 'Fare', 'Minor', 'SurnameAllDied', 'SurnameOneLived', 'FamilySize', 'female', 'male', 'D', 'E')
0.8262711864406779
